GRA2PES Demo
================

This notebook demonstrates multiple components of the GRA2PES framework, including:
- **Base data loading**: Loading the base data given the GRA2PES configuration.
- **Regridding**: Regridding the base LCC data to the target Lat/Lon grid. 
- **Regridded data loading**: Loading the regridded data given the GRA2PES configuration.
- **Basic processing**: Applying things like unit conversion etc. to the regridded data.
- **Emissions visualization**: Visualizing the regridded data using matplotlib.
- **Ratio visualization**: Visualizing the ratio of two regridded datasets.

To be able to run this notebook, you will have to have downloaded the GRA2PES base data using the gra2pes_base_creator.py script, as well as have a regridded dataset using the gra2pes_regrid.py script. 

**NOTE**: These datasets are quite large and require a lot of RAM to load in memory. I have only run this notebook on a CHPC cluster with lots of memory (>100GB). Be aware that running this notebook in other instances may result in memory errors.


### Setup
Ensure that the parameters in inventories/gra2pes/gra2pes_config.py are set correctly for your use case.

In [ ]:
#Import necessary libraries
import os
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe
import sys 
import matplotlib
import matplotlib.pyplot as plt
import pyproj
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
sys.path.append('..')
from inventories.gra2pes import gra2pes_utils , gra2pes_config, gra2pes_regrid
from utils import gen_utils, datetime_utils, xr_utils, plot_utils, df_utils

# Base Data Loading/Examining
"Base" data referes to the data that is loaded from the dataset downloaded and organized by inventories/gra2pes/gra2pes_base_creator.py. These are the extracted tar.gz files from either the NOAA FTP or HTTPS servers. 

In [ ]:
config = gra2pes_config.Gra2pesConfig() # Load the configuration 
extra_ids = 'methane' #define any "extra" ids that are not included in the default set of species
specs = ['CO2','CO','HC01']#,'HC02','HC14','NH3','NOX','SO2']

BGH = gra2pes_utils.BaseGra2pesHandler(config,specs = specs, extra_ids = extra_ids) #create the handler object

sector = 'AG' # Define the sector of interest
year = 2021 # Define the year of interest
month = 1 # Define the month of interest
day_type = 'weekdy' # Define the type of day (e.g., 'weekdy' for weekdays, 'satdy' for Saturdays, 'sundy' for Sundays)
base_ds = BGH.load_fmt_fullday(sector, year, month, day_type, check_extra=False) # Load the full day dataset for the specified sector, year, month, and day type
base_ds # Display the dataset

# Regridding
This is an example of what happens in the gra2pes_regrid.py script to transform the base data from the LCC grid to the target Lat/Lon grid. Configuration for the regridding is set in inventories/gra2pes/gra2pes_config.py in the Gra2pesRegridConfig class. 

**NOTE**: This will take a bit of time to run -- for the example data loaded in the cell above, it takes about 3 minutes. 

In [ ]:
regrid_config = gra2pes_config.Gra2pesRegridConfig(config) # Load the regrid configuration
gra2pes_regridder = gra2pes_utils.Gra2pesRegridder(regrid_config) # Create the regridder object
base_ds = gra2pes_regrid.sum_on_dim(base_ds, dim ='zlevel') #Sum over the vertical levels to get a 2D dataset for much faster regridding
regridded_ds = gra2pes_regridder.regrid(base_ds) # Regrid the dataset to the desired grid
regridded_ds = gra2pes_regrid.slice_extent(regridded_ds,extent={'lon_min': -113, 'lon_max': -111, 'lat_min': 40, 'lat_max': 42}) # Slice the dataset to a specific extent
regridded_ds # Show the regridded dataset

# Working with Regridded Data
This section demonstrates how to load the data that was regridded and saved using the gra2pes_regrid.py script. This allows the user to load the necessary data for further processing, visualization, or analysis.

In [ ]:
# Load the cgeneral configuration for the gra2pes
config = gra2pes_config.Gra2pesConfig() 

# Define the regrid ID, which corresponds to the grid resolution. This is definde in the configuration file.
regrid_id = '0.025x0.025' 

# Define the date range for which you want to open the regridded dataset
dtr = datetime_utils.DateTimeRange('2021-01-01','2021-12-31 23:59',tz = 'UTC')

# Create the regridded handler object
rgh = gra2pes_utils.RegriddedGra2pesHandler(config,regrid_id) 

# Open the regridded dataset for the specified date range and sectors
regridded_ds = rgh.open_ds_inrange(dtr,sectors='all')

# Open the grid area dataset to get the cell area for the regridded grid, this should have been created
# during the regridding process if run using the regrid script provided in the repository.
gca = xr.open_dataset(os.path.join(rgh.regridded_path,'details','grid_out_area.nc'))['cell_area']

In [ ]:
# The regridded dataset is stored as an xarray dataset, separated by lat, lon, year, month, daytype (satdy, sundy, weekdy), 
# and sector as it is defined in the original GRA2PES tar files. 

regridded_ds # Show the regridded dataset

We can also rework the time dimension to be more user-friendly and subsettable. This turns the multiple dimensions representing time (year, month, day_type, hour) into a single time dimension that can be easily indexed.

**NOTE**: This can take quite a while to run depending on the extent of the data and the number of variables being processed. It is recommended to run this on a slice of the data. In this example, we are loading the dataset I regridded for the Salt Lake Valley using the gra2pes_regrid.py script with an extent of 40.0 to 42.0 latitude and -113.0 to -111.0 longitude.

In [ ]:
# Rework the dataset to a more convenient format. This can take a bit of time to run (~3 minutes for full dataset)
ds = rgh.rework_ds_dt(regridded_ds[['CO2','CO','HC01']]) #Subset the dataset to 3 species so it doesn't take up all the RAM

In [ ]:
#View the reworked dataset
ds

Now we can easily index the data by time, which is a single dimension. We can also slice by lat/lon and select specific sectors and data variables for analysis. We can also apply unit conversions to the data variables as needed. Examples are given below. 

In [ ]:
# Trim to lat/lon extent:
slv_extent = {'lon_min':-112.15,
                  'lon_max':-111.7,
                  'lat_min':40.351,
                  'lat_max':41.2} 

slv_ds = xr_utils.trim_ds_to_extent(ds,slv_extent)
slv_gca = xr_utils.trim_ds_to_extent(gca,slv_extent)

In [ ]:
# Convert units
molar_masses = {'CO2':44.01, 'CO':28.01, 'HC01':16.04} # Define molar masses for conversion
unit_converter = xr_utils.UnitConverter() # Create an instance of the UnitConverter class
slv_ds = unit_converter.convert(slv_ds, 'km^-2', 'm^-2') # Convert units from km^-2 to m^-2
slv_ds = unit_converter.convert_mole_to_g(slv_ds, molar_masses) # Convert moles to grams using the molar masses defined above
slv_ds = unit_converter.convert(slv_ds, 'g', 'metric_Ton') # Convert grams to metric tons

# Convert to absolute units of mass per grid cell using the grid cell area
absolute_slv_ds = unit_converter.convert_m2_to_gridcell(slv_ds, slv_gca) 

In [ ]:
absolute_slv_ds

## Emissions Visualization
Using the regridded, retimed, and unit-converted data, we can visualize the emissions data using matplotlib. I haven't yet made these plotting function modular, so the defined parameters are in the cells below, and can be altered as needed.

In [ ]:
# Define the plot characteristics 

# The dataset to plot. This is where you can specify any sums/averages from the dataset. In this case, I am plotting
# the sum of the entire year, but this can be changed to any other aggregation you want.
plot_ds = absolute_slv_ds.sum(dim = 'datetime', keep_attrs=True)
plot_ds = unit_converter.update_all_units(plot_ds, 'hr^-1','yr^-1') # Convert the units to per year since we summed over the year. 

# The map extent for the plot (usually slightly larger than the data extent)
map_extent={'lon_min':-112.22,
            'lon_max':-111.65,
            'lat_min':40.35,
            'lat_max':41.3}  

# The species to plot, this should be one of the species in the dataset
species = 'CO2' 

# The sector to plot, 'total' is an included option that is the sum of all sectors
sector = 'total' 

# The colormap to use for the plot
cmap = 'Blues' 

In [ ]:
# Create the figure
fig = plt.figure(figsize = (8,8)) # Define the figure size
labsize = 20  # Define the label sizes
proj = ccrs.PlateCarree() # Define the projection for the map
ax = plt.axes(projection = proj) # Create the axes for the plot with the defined projection

# Set the extent of the map
ax.set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)
request = cimgt.GoogleTiles(style='satellite') # Use Google satellite tiles for the background map
scale = 12.0  # Define the scale for the background map, higher values give more detail (also take longer to load)
ax.add_image(request,int(scale)) # Add the background map to the axes

# Define the max value for the colorbar. This is set to 90% of the maximum value in the dataset to avoid 
# having very high values skewing the colorbar.
max_value = plot_ds[species].max().values
vmax_value = max_value*0.9

# Plot the data on the map
map = plot_ds[species].sel(sector=sector).plot.pcolormesh('lon','lat',ax = ax,alpha=0.6,cmap=cmap,add_colorbar=False,edgecolors = (0,0,0,0.1),linewidth = 0.2,vmax = vmax_value)

# Add the colorbar to the plot
cbar = plt.colorbar(map,fraction=0.05,pad = 0.02)
cbar.set_label(label = plot_ds[species].sel(sector=sector).units, size = labsize)
cbar.ax.tick_params(labelsize = labsize)

plt.show()

# Ratio Visualization
This section demonstrates how to visualize the ratio between two species at the grid cell level. 

In [ ]:
# Define the ratio dataset and convert the units
ratio_ds = absolute_slv_ds.sel(sector='total').sum(dim = 'datetime', keep_attrs=True)
ratio_ds = unit_converter.update_all_units(ratio_ds, 'hr^-1','yr^-1') # Convert the units to per year since we summed over the year. 

# Define the inventory ratios object
ir = xr_utils.InventoryRatios(ratio_ds,molar_masses)

In [ ]:
# Define the parameters for the ratio plot
numerator = 'HC01' # The numerator species for the ratio
denominator = 'CO2' # The denominator species for the ratio

# These quantile filters are used to avoid having very low values skewing the ratio plot
num_quantile_filter = 0.25 # The quantile filter for the numerator, this is used to filter out low values in the numerator
denom_quantile_filter = 0.5 # The quantile filter for the denominator, this is used to filter out low values in the denominator

# Get the ratio of the numerator to the denominator using the defined quantile filters. 
ratio_ds = ir.get_gc_ratio(numerator, denominator, 
                        num_quantile_filter = num_quantile_filter, 
                        denom_quantile_filter = denom_quantile_filter) 

# Add the CO2 equivelents to the ratio dataset to use for filtering plot cells
ratio_ds['CO2_eq_sum'] = ratio_ds['CO2_eq_sum'].fillna(0)

ratio_id = f'{numerator}_{denominator}'.lower() # Define the ratio ID for the plot

In [ ]:
# Make the plot

da = ratio_ds[ratio_id]*1000
alpha_var = 'CO2_eq_sum'
min_alpha = 0.3
alphas = ratio_ds[alpha_var].values
alphas = (alphas - alphas.min())/(alphas.max()-alphas.min())*(1-min_alpha)+min_alpha

labsize = 40
proj = ccrs.PlateCarree()

fig = plt.figure(figsize = (12,12))
ax = plt.axes(projection = proj)
ax.set_extent([map_extent['lon_min'],map_extent['lon_max'],map_extent['lat_min'],map_extent['lat_max']],crs=proj)
request = cimgt.GoogleTiles(style='satellite')
scale = 12.0 # prob have to adjust this
ax.add_image(request,int(scale))

map = da.plot.pcolormesh('lon','lat',ax = ax,alpha=alphas,cmap='viridis',add_colorbar=False,edgecolors = (0,0,0,1),linewidth = 2,vmin = 0,vmax=10)#,vmin=0,vmax = 10)
da.plot.pcolormesh('lon','lat', ax = ax,facecolors='none', edgecolors=(0, 0, 0, 1), linewidth=2,add_colorbar=False)

cbar = plt.colorbar(map,fraction=0.045,pad = 0.02,location = 'right')
cbar.set_label(label = f'{ratio_id} ratio (permille)',size = labsize)
cbar.ax.tick_params(labelsize = labsize)


plt.show()